# Formelgenerator

Für "normale" Formeln und für KNF's in Hornform.



In [145]:
# Formelerzeugung für normale Formeln
import random
import math

# Define some helpful symbols
sym_or =    '\u2228'  # ∨
sym_and =   '\u2227'  # ∧
sym_imply = '\u27f6'  # ⟶ oder '\u2192' = →
sym_neg   = '\u00AC'  # ¬
sym_equiv = '\u27f7'  # ⟷ oder '\u2194' = ↔

ATOMS = ['A','B','C','D','E','F','G','H'] 

TYP_OP   = 1
TYP_ATOM = 0

all_ops = [sym_or,sym_and,sym_imply,sym_neg,sym_equiv] 

# Man kann diese Liste auch mit doppelten oder gar dreifachen Operatoren
# befüllen, dann erhalten diese eine höhere Verwendungswahrscheinlichkeit.
# Denkbar wäre z.B. öfter nicht, oder, und, seltener ->, <->, etwas so:

# all_ops = [sym_neg,sym_neg,sym_neg,sym_neg,sym_or,sym_or,sym_and,sym_and,sym_imply,sym_equiv]
# Das würde - im Mittel über viele Formeln - zu 40% NOT, 20% OR, 20% AND, 10% IMPLY und 10% <->
# führen

# Man könnte stattdessen auch mit Listen von Operator"instanzen" arbeiten,
# die man nach der Verwendung aus der Liste entfernt (dann könnte man
# GENAU vorgeben, welche Operatoren wie oft verwendet werden sollen, in
# dem man eine Liste mit genau 8, 10, 12 Operatoren "per Hand" erstellt).
# Erscheint aber unnötig. 

# Die Idee ist "straight-forward": wir erzeugen direkt einen Baum

class Formelbaum(object):
    def __init__(self, typ, content):
        self.typ      = typ
        self.content  = content 
        self.children = [] # A list of trees
    
    def addchild(self,tree):
        self.children.append(tree)
    
    def show(self,level=0):
        if level: 
            print((level-1)*'    ','|')
            print((level-1)*'    ','---',self.content)
        else:
            print(self.content)
        for c in self.children:
            c.show(level+1)
        
    def als_formel(self,level=0):
        result = self.content
        if (self.typ == TYP_OP):            
            if (self.content == sym_neg):
                # unärer Operator Negation
                result = result + self.children[0].als_formel(level+1)
            else:
                # binärer Operator
                result = self.children[0].als_formel(level+1) \
                            + " " + result + " " \
                            + self.children[1].als_formel(level+1)
                if level:
                    result = "(" + result + ")"
        return result
    
    ### Alles in dieser Klasse ab hier braucht man nur für die Hornformeln
    ### Das oben kann man für die normalen Formeln so auch in Java übernehmen!
    
    # Das benutzen wir erst später für die Hornformeln, ist aber an sich 
    # generell, sollte also für jede "korrekte" Formel funktionieren
    def als_horn_formel(self,level=0):
        if (self.typ == TYP_OP):            
            if (self.content == sym_neg):
                # unärer Operator Negation
                result = self.content + self.children[0].als_horn_formel(level+1)
            else:
                # Iteriere über alle Kinder
                res_list = []
                for c in self.children:
                    res_list.append(c.als_horn_formel(level+1))                
                result = res_list[0]
                for r in res_list[1:]:
                    result += " " + self.content + " " + r                            
                if level:
                    result = "(" + result + ")"
        else:
            result = self.content # an atom!
        return result

    # Helpermethode, die eine Disjunktion von Literalen als
    # Liste von Literal-Strings repräsentiert (eine Art Klauselform, die
    # man daraus leicht erzeugen könnte)
    def klausel(self):
        if (self.typ == TYP_OP):
            if self.content == sym_neg: # Negatives Literal
                res = ['-' + self.children[0].content]
            else:
                # nicht allgemein sinnvoll hier, für OR (ev. AND) gebaut
                res = []
                for c in self.children:
                    res += c.klausel()
        else: # an atom
                res = [self.content]
        return res
    
    # Wir für die Horn-Formel-Geschichte genutzt, geht so nur auf
    # KNF oder DNF-Formeln
    def add_unique_child(self,tree):
        tr_kls = tree.klausel()
        tr_kls.sort() # watch it, list.sort() does NOT return the sorted list (but None...), crazy
        # so, we can NOT write c_kls.sort() == tr_kls.sort()...sad
        # We could use "sorted(list)" instead, but we may want to reuse the sorted list later
        # ... and keeping tree sorted may reduce workload (marginally, as merge sort is used)
        for c in self.children:
            c_kls = c.klausel()
            c_kls.sort()
            if tr_kls == c_kls:
                # print("Duplicate found") # this happens indeed sometimes
                return False
        self.addchild(tree)
        return True
        

# Wir nutzen hier "choice" für den Moment, ändern wir noch
# random.choice(seq)
# Return a random element from the non-empty sequence seq. If seq is empty, raises IndexError.
# from: https://docs.python.org/3/library/random.html

def chose_operator(op_set):
    return random.choice(op_set)

''' The following is not used!!!
def chose_atom(anzahl,must_atoms=[]):
    l = len(must_atoms)
    # the if...-branch ensures that, first, atoms from the must_use_list
    # are used. This implies, that the formulas are not truely random
    # anymore, as we use a left-first-recursion for binary operators,
    # which means that it is less likely to have duplicates to the
    # left of the root than to its right. This is not a problem, as long
    # as we don't need (for whatever reason) truely random formulas
    # If we "don't care" whether as many different atoms as possible,
    # given the number of atom types to be used, are used, the if-branch
    # can be removed (currently, we DO care, so it should stay ;)
    if l:
        pos = random.choice(range(l)) # Wähle einen int aus 0..(l-1) aus
        atom = must_atoms[pos] # hole das Atom raus
        del must_atoms[pos] # Lösche es in must_atoms
        return atom 
        # ANMERKUNG: Wenn man wollen würde, dass sich die Atome "schön" abwechseln 
        # und es nicht zu (A -> A) kommen soll, dann könnte man nach jeder
        # Leerung der must_atoms die auf die Startauswahl zurücksetzen.
        # Dann könnte zwar immer noch zufällig das in der Vorrunde zuletzt
        # gewählte erneut entnommen werden, aber dann geht es sicher nicht
        # mit einem weiteren Duplikat weiter. Das könnte man hier einbauen!    
        # (wurde so gemacht, s. unten)
    # ...all must_atoms have been used already
    return random.choice(ATOMS[:anzahl])
'''

# Die folgende Variante von chose_atom "zirkuliert" durch die Atome, die verwendet werden sollen
# und stellt sicher, dass es keine zu große Häufung der Auswahl des gleichen Atoms
# hintereinander gibt. 
# Beispiel: A,B,C sollen verwendet werden
# Wenn wir 6 Verwendungen von Atomen haben, wir die Blattebene von links nach rechts
# zuerst eine Permutation von A,B,C sein, dann wieder eine von A,B,C, denkbar also
# z.B. A,B,C,A,B,C oder B,C,A,A,B,C, es kann also weiter Doppelungen geben an den
# Übergängen, aber keine Teilsequenzen, die aus 3 oder mehr wiederholten gleichen
# Buchstaben bestehen - und es ist sicher gestellt (in dem Beispiel) das vor dem 
# nächsten A zunächst B und C (in einer beliebigen Reihenfolge) kommen.
#
# Die Version oben stellt nur sicher, dass wir (von links nach rechts in den Blättern)
# mit einer A,B,C-Permutation beginnen, danach geht es rein zufällig weiter. Das
# dürfte ziemlich der jetzigen Lösung in Java entsprechen. Die neue Version ist
# "realistischer" (ich würde in der Klausur so etwas wie (B v (B -> B))) sicher 
# vermeiden - auch wenn es an sich nicht uninteressant ist (das ist übrigens gültig, klar)

def chose_atom(anzahl,must_atoms=[]):
    if not must_atoms: # falls die Liste gerade leer ist (len(must_atoms) == 0 ginge auch)
        for atom in ATOMS[0:anzahl]:
            must_atoms.append(atom)
    
    l = len(must_atoms)
    pos = random.choice(range(l)) # Wähle einen int aus 0..(l-1) aus
    atom = must_atoms[pos] # hole das Atom raus
    del must_atoms[pos] # Lösche es in must_atoms
    return atom 


# wir teilen die Anzahl der verbleibenden Operatoren in zwei Teile
# für links und rechts (nur bei binären Operatoren relevant)

def determine_ratio(anzahl):
    ratio = random.random() # 0 <= ratio < 1
    split = round(anzahl * ratio)
    left = split
    right = anzahl - split
    # print(anzahl,split,ratio*anzahl,": (",left,",",right,")")
    return left,right
    
    
# In der ersten Version verwenden wir als Vorgabe die Anzahl der Operatoren, die wir 
# gern in dem Baum haben möchten und die Anzahl verschiedener Atome. Wir können auch
# noch die Operatormenge vorgeben
# must_atoms enthält die Atome, die wir mindestens einmal (falls möglich) verwenden
# möchten (die wird von allen rekursiven Aufrufen gemeinsam gepflegt, muss also eine
# Referenz sein auf ein Objekt "außen" und keine Kopie) 

def generator(atomtypen_anzahl,operatoren_anzahl,must_atoms=[],op_set=all_ops):
    if operatoren_anzahl > 0:
        op = chose_operator(op_set) # Operator wählen
        tr = Formelbaum(TYP_OP,op) # Baum erzeugen mit besetzter Wurzel
        # Verhältnis bestimmen für die Anteile an verbleibenden
        # Operatoren für links und rechts
        if op != sym_neg: # Keine Negation
            op_count_left,op_count_right = determine_ratio(operatoren_anzahl-1)
        else:
            op_count_left,op_count_right = operatoren_anzahl-1,0
        # Ein Kind erzeugen
        tr.addchild(generator(atomtypen_anzahl,op_count_left,must_atoms,op_set))
        # Falls es keine Negation ist, ein weiteres Kind erzeugen
        if op != sym_neg:  # Negation, also einstellig
            tr.addchild(generator(atomtypen_anzahl,op_count_right,must_atoms,op_set))
    else:
        tr = Formelbaum(TYP_ATOM,chose_atom(atomtypen_anzahl))
    return tr

for i in range(100):
    # print(generator(3,4).show())
    atomtyp_count = 4
    print("Formel ",i,":",generator(atomtyp_count,14).als_formel())




Formel  0 : ((B ⟷ (D ∨ ¬(¬C ⟶ A))) ∧ (((A ∨ C) ⟶ ¬B) ∧ (D ⟶ A))) ⟶ ¬(C ∧ B)
Formel  1 : ¬(D ⟶ D) ⟶ (¬(A ⟷ B) ∨ ((C ⟶ D) ⟶ ((C ∨ (A ∧ B)) ∨ ¬((D ∨ A) ⟷ B))))
Formel  2 : ¬¬(((C ∧ ¬(B ⟶ ((D ⟶ C) ⟷ ((A ⟶ C) ∧ D)))) ∧ (A ∨ (B ∧ (B ⟶ D)))) ∨ C)
Formel  3 : ¬A ⟶ (((((((C ∧ D) ∧ ((B ⟶ A) ⟶ B)) ⟷ ¬A) ⟶ ¬(D ⟷ C)) ⟶ B) ⟶ A) ⟶ C)
Formel  4 : D ∧ (¬(B ∨ (¬(C ∧ A) ∧ ((D ∧ A) ⟶ (D ∧ B)))) ⟷ ((C ⟶ C) ⟷ (¬B ⟷ D)))
Formel  5 : ¬(((A ⟶ B) ⟷ ¬D) ∨ (A ∧ (C ⟷ (¬B ⟷ D)))) ∧ (¬(A ⟷ (C ⟷ D)) ⟶ C)
Formel  6 : ((B ∧ A) ∨ (C ⟶ A)) ⟷ ((D ⟶ B) ⟶ ¬¬(((C ⟶ A) ∨ D) ⟶ (((B ⟶ B) ⟷ A) ⟷ D)))
Formel  7 : ((¬¬(C ⟶ D) ⟷ (C ∨ B)) ∧ (A ∧ (D ∧ B))) ⟶ (¬C ∧ ¬(A ⟶ (B ⟶ A)))
Formel  8 : (((C ∨ D) ⟷ ((B ∨ A) ⟶ D)) ⟶ ((C ∨ ¬A) ⟶ C)) ⟷ (¬(D ⟶ B) ⟷ ¬(C ⟶ D))
Formel  9 : ¬(A ⟶ B) ∨ (((D ∧ ¬B) ∨ (C ∧ A)) ∨ ¬((D ⟷ ((B ∨ C) ∧ ¬A)) ⟶ B))
Formel  10 : (((D ∨ A) ∨ C) ⟷ ((¬C ∨ D) ∧ ((A ∧ ¬B) ∧ (A ∨ C)))) ∨ (B ∧ ((D ∧ D) ⟷ C))
Formel  11 : (((B ⟶ A) ∨ (B ⟷ D)) ⟷ ((A ⟶ C) ⟷ B)) ∧ ((((D ∨ A) ⟶ ¬C) ⟶ (¬D ⟷ B)) ∨ C)
Formel  12 : (¬(¬A ⟶ (A ⟷ B))

In [125]:
# Jetzt bauen wir noch etwas für unsere Hornformeln als Vorgabe für die Implikationsform
# Die Bäume kann man so lassen (die sind schon allgemein durch die Kinderliste)

def pick(picklist):
    if not picklist: 
        print("WARNING!!")
        return 'A' # should never happen!
    l = len(picklist)
    pos = random.choice(range(l)) # Wähle einen int aus 0..(l-1) aus
    atom = picklist[pos] # hole das Atom raus
    del picklist[pos] # Lösche es in picklist
    return atom 
    
# Funktioniert nicht, wenn man eine 1 erlaubt als Länge und diese gewähle würde.
def horn_generator(and_range,or_max):
    # Geniere und-Knoten
    tr_and = Formelbaum(TYP_OP,sym_and)
    ad = random.choice(and_range) # and_range ist z.B. [4,5,6], davon wählen wir einen aus
    # ad steht für "Anzahl Disjunktionen"
    # Wir müssen sicherstellen, dass wir mit einer recht hohen Wahrscheinlichkeit ein einzelnes
    # positives Atom in einer Disjunktion haben.
    choice_list = []
    for i in range(1,or_max+1):
        for j in range(1,or_max+1-i):
            choice_list.append(i)
    # Generate or's            
    for i in range(0,ad):
        candidates = ATOMS[:or_max] # Neue Kandidaten für jedes OR
        size = random.choice(choice_list)
        has_positive = False # es gibt noch kein positives Literal im OR 
        if size > 1:
            tr_or = Formelbaum(TYP_OP,sym_or)
        for j in range(0,size):
            # Choose Atom
            cand = pick(candidates)
            is_positive = False
            if not has_positive:
                if random.random() < 0.7:
                    is_positive = True
            if not is_positive:
                # Negativ
                tr_child = Formelbaum(TYP_OP,sym_neg)
                tr_child.addchild(Formelbaum(TYP_ATOM,cand))
            else:
                # Positiv
                tr_child = Formelbaum(TYP_ATOM,cand)
                has_positive = True
            if size == 1:  
                tr_and.addchild(tr_child)
            else:
                tr_or.addchild(tr_child)
        if size > 1:
            tr_and.addchild(tr_or)
    return tr_and


for i in range(10):
    print("Formel ",i,":",horn_generator([4,5,6],4).als_horn_formel())
    # Kontrolle
    # print("Formel ",i,":",horn_generator([4,5,6],4).show())


Formel  0 : D ∧ (¬B ∨ C ∨ ¬A) ∧ (D ∨ ¬C ∨ ¬B) ∧ (D ∨ ¬B ∨ ¬C) ∧ (¬B ∨ C ∨ ¬A)
Formel  1 : (B ∨ ¬A ∨ ¬D) ∧ ¬C ∧ (¬B ∨ D) ∧ (¬D ∨ ¬C ∨ ¬A) ∧ A
Formel  2 : (A ∨ ¬D ∨ ¬B) ∧ (C ∨ ¬D ∨ ¬A) ∧ ¬A ∧ (D ∨ ¬A) ∧ B
Formel  3 : C ∧ ¬A ∧ A ∧ (A ∨ ¬D) ∧ B ∧ A
Formel  4 : (D ∨ ¬C) ∧ C ∧ (C ∨ ¬B) ∧ ¬C ∧ (B ∨ ¬C ∨ ¬A) ∧ (¬A ∨ ¬C ∨ B)
Formel  5 : (D ∨ ¬A ∨ ¬C) ∧ C ∧ (B ∨ ¬C ∨ ¬A) ∧ (D ∨ ¬C ∨ ¬A) ∧ ¬C
Formel  6 : (D ∨ ¬C) ∧ (B ∨ ¬A) ∧ (D ∨ ¬A) ∧ (C ∨ ¬A)
Formel  7 : (C ∨ ¬B) ∧ D ∧ (C ∨ ¬B ∨ ¬D) ∧ ¬B ∧ (D ∨ ¬C) ∧ C
Formel  8 : B ∧ (D ∨ ¬B ∨ ¬C) ∧ (¬A ∨ D) ∧ C
Formel  9 : (¬B ∨ ¬D) ∧ C ∧ (C ∨ ¬B) ∧ (D ∨ ¬B)


In [147]:
# Ein Problem, das wir bei beiden Erzeugungen haben (bei der allgemeinen stört es
# aber nicht wirklich, kann ja vorkommen!), ist, dass wir OR-Klauseln doppelt
# erzeugen. Das fällt vor allem auf, wenn es mehrfach die gleiche einzeln Klausel
# wird. Das ist nicht so leicht zu vermeiden, vor allem dann nicht, wenn man
# sicherstellen will, dass man nicht ewig würfelt (man kann die Versuche
# mitzählen und dann nach z.B. 10 Versuchen doch etwas Doppeltes akzeptieren,
# ist ja semantisch nicht falsch.

# Erstellen wir einmal eine Version, die so arbeitet (wie wird das in unserer Resolution
# gemacht? Vielleicht liegt da ein Problem, oder achten wir darauf gar nicht?)

# Die Hauptfunktionalität dafür ist oben schon in der Klasse mit den Bäumen
# untergebracht, s. oben!

# Hiermit kontrollieren wir, wie die Verteilung der Anzahlen von Literalen
# in den OR-Teilformeln aussieht
def create_choice_list_1(max_count):
    choice_list = []
    for i in range(1,max_count+1):
        for j in range(1,max_count+2-i):
            choice_list.append(i)
    # print(choice_list) 
    return choice_list
    # Example: für max_count = 5 wird
    #  [1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5] 
    # produziert
    
# Könnte man auch anders handhaben, z.B. so:
def create_choice_list_2(max_count):
    choice_list = [1,1,1,1]
    for i in range(1,3+1):
        for j in range(0,3):
            choice_list.append(i)
    for i in range(4,max_count):
        for j in range(0,2):
            choice_list.append(i)            
    # print(choice_list) 
    return choice_list
    # Example: für max_count = 5 wird
    #  [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5] 
    # produziert

create_choice_list = create_choice_list_2

def new_horn_generator(and_range,or_max,atom_max):
    # Geniere und-Knoten
    tr_and = Formelbaum(TYP_OP,sym_and)
    ad = random.choice(and_range) # and_range ist z.B. [4,5,6], davon wählen wir einen aus
    # ad steht für "Anzahl Disjunktionen"
    # Wir müssen sicherstellen, dass wir mit einer recht hohen Wahrscheinlichkeit ein einzelnes
    # positives Atom in einer Disjunktion haben.
    choice_list = create_choice_list(or_max)    
    # Generate or's            
    for i in range(0,ad):       
        size = random.choice(choice_list)
        found = False
        safety_count = 20
        while safety_count and not found:
            candidates = ATOMS[:atom_max] # Neue Kandidaten für jeden Versuch,
            # eine OR-Teilformel zu bauen
            has_positive = False # es gibt noch kein positives Literal im OR 
            if size > 1:
                tr_toadd = Formelbaum(TYP_OP,sym_or)            
            for j in range(0,size):
                # Choose Atom
                cand = pick(candidates)
                is_positive = False
                if not has_positive:
                    if random.random() < 0.7:
                        is_positive = True
                if not is_positive:
                    # Negativ
                    tr_child = Formelbaum(TYP_OP,sym_neg)
                    tr_child.addchild(Formelbaum(TYP_ATOM,cand))
                else:
                    # Positiv
                    tr_child = Formelbaum(TYP_ATOM,cand)
                    has_positive = True
                if size == 1:  
                    tr_toadd = tr_child
                else:
                    tr_toadd.addchild(tr_child)
            # Try to add it to the and-node
            if tr_and.add_unique_child(tr_toadd):
                found = True
            else:
                safety_count -= 1
        # We accept the last candidate if we tried safety_count times
        if not found:
            print("Duplicate accepted!!")
            tr_and.addchild(tr_toadd) # this always exists, unless we set
            # safety_count to 0, which would be outright stupid
    return tr_and


for i in range(20):
    print("Formel ",i,":",new_horn_generator(and_range=[5,6,7],or_max=5,atom_max=5).als_horn_formel())


Formel  0 : (E ∨ ¬C) ∧ (A ∨ ¬B ∨ ¬D ∨ ¬C) ∧ (B ∨ ¬D ∨ ¬A ∨ ¬C) ∧ E ∧ A ∧ B
Formel  1 : E ∧ ¬A ∧ ¬D ∧ A ∧ C ∧ (D ∨ ¬C ∨ ¬A ∨ ¬E) ∧ (A ∨ ¬B ∨ ¬D ∨ ¬C)
Formel  2 : (B ∨ ¬A) ∧ (D ∨ ¬A ∨ ¬B) ∧ (¬B ∨ E ∨ ¬C ∨ ¬A) ∧ C ∧ ¬A ∧ (E ∨ ¬B)
Formel  3 : (D ∨ ¬C) ∧ B ∧ E ∧ ¬D ∧ D ∧ ¬A
Formel  4 : ¬C ∧ (B ∨ ¬E ∨ ¬A) ∧ C ∧ (E ∨ ¬A) ∧ (¬B ∨ A) ∧ D ∧ (B ∨ ¬D ∨ ¬E ∨ ¬A)
Formel  5 : (C ∨ ¬B ∨ ¬E ∨ ¬A) ∧ (C ∨ ¬B ∨ ¬D ∨ ¬A) ∧ ¬B ∧ A ∧ (¬B ∨ E)
Formel  6 : (E ∨ ¬B) ∧ ¬A ∧ C ∧ (C ∨ ¬E ∨ ¬A ∨ ¬D) ∧ (C ∨ ¬D) ∧ (A ∨ ¬E ∨ ¬B ∨ ¬C) ∧ B
Formel  7 : (¬D ∨ ¬C) ∧ (C ∨ ¬B) ∧ (A ∨ ¬C ∨ ¬B) ∧ (¬C ∨ A ∨ ¬B ∨ ¬D) ∧ (¬D ∨ A ∨ ¬E ∨ ¬B) ∧ (E ∨ ¬B ∨ ¬D)
Formel  8 : A ∧ (C ∨ ¬A) ∧ C ∧ (E ∨ ¬A) ∧ ¬D ∧ ¬A
Formel  9 : (¬A ∨ E) ∧ ¬A ∧ (C ∨ ¬A ∨ ¬E) ∧ (¬D ∨ E ∨ ¬B) ∧ C ∧ B ∧ A
Formel  10 : ¬A ∧ D ∧ A ∧ (¬D ∨ ¬A ∨ B ∨ ¬E) ∧ ¬D ∧ B
Formel  11 : (A ∨ ¬B ∨ ¬C ∨ ¬D) ∧ C ∧ (E ∨ ¬D ∨ ¬A ∨ ¬C) ∧ (¬E ∨ ¬B ∨ A ∨ ¬D) ∧ (C ∨ ¬A ∨ ¬E ∨ ¬B) ∧ ¬E
Formel  12 : (¬C ∨ B ∨ ¬E ∨ ¬D) ∧ ¬E ∧ (C ∨ ¬A) ∧ (E ∨ ¬A ∨ ¬D ∨ ¬B) ∧ (¬D ∨ A)
Formel  13 : D ∧ E ∧ A ∧